# Generative Adversarial Networks 
by Alexander Lercher

## Tensorflow Setup

In [ ]:
import tensorflow as tf

# Execute this to avoid internal tf error
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Generator for Training Data 

In [ ]:
# Simple Dataset (MNIST Handwritten Digits)
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
# Normalize the image pixels to [-1, 1] according to paper
x_train = (x_train - 255/2.) / (255/2.) 

mnist_data = tf.data.Dataset.from_tensor_slices(x_train).shuffle(x_train.shape[0]).batch(128)

def mnist_generator():
    return mnist_data

In [ ]:
from training_images.training_data_provider import TrainingDataProvider
provider = TrainingDataProvider()

facial_image_generator = provider.get_all_training_images_in_batches_from_disk

## Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks
Alec Radford, Luke Metz, Soumith Chintala

https://arxiv.org/abs/1511.06434

In [ ]:
from gan import DCGAN_Faces

gan = DCGAN_Faces()

gan.set_training_data(facial_image_generator)

In [ ]:
gan.train(epochs=50)

In [ ]:
gan.generate_gif()
gan.export()

## High-Resolution Deep Convolutional Generative Adversarial Networks
J. D. Curtó, I. C. Zarza, Fernando de la Torre, Irwin King, Michael R. Lyu

https://arxiv.org/abs/1711.06491

In [ ]:
from gan import HR_DCGAN_Faces

gan = HR_DCGAN_Faces()

gan.set_training_data(facial_image_generator)

In [ ]:
gan.train(epochs=50)

In [ ]:
gan.generate_gif()
gan.export()

## Visualization and Debug Output

In [ ]:
import matplotlib.pyplot as plt

img = gan.generate()
plt.imshow((img[0, :,:,0] ), cmap='gray')
plt.show()

In [ ]:
import numpy as np
noise = np.random.normal(0, 1, (5, 100))
batch_fake = gan.generator.predict(noise)

res = gan.discriminator.predict(batch_fake)
res

In [ ]:
res = gan.discriminator.predict(x_train[0:5])
res